In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
pip install pandasql

  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26772 sha256=9358dc345ac901b49e849ba736fe4858f8d188f38b9648ebab242e0869fc2f6d
  Stored in directory: /root/.cache/pip/wheels/e9/bc/3a/8434bdcccf5779e72894a9b24fecbdcaf97940607eaf4bcdf9
Successfully built pandasql


In [ ]:
pip install SQLAlchemy==1.4.46

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.4 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.32
    Uninstalling SQLAlchemy-2.0.32:
      Successfully uninstalled SQLAlchemy-2.0.32
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.46 which is incompatible.


In [ ]:
import pandas as pd
import os
from pandasql import sqldf


#os.chdir('..')

# define a lambda to make queries easier
pysqldf = lambda q: sqldf(q, globals())
#employees = pd.read_excel("Employees Excel Sheet.xlsx", sheet_name='employees.csv')
employees = pd.read_excel("/content/drive/My Drive/SQL_Assignment/Employees.xlsx")

# Import orders data - Matt G, Jason T, Matt W, and La Shanda H go to sushi lunch
# taking a potential client with them

# orders <- read.csv("~/orders.csv")
orders = pd.read_excel("/content/drive/My Drive/SQL_Assignment/Orders.xlsx")

In [ ]:
employees

,id,lastname,firstname,gender
0,1,g,matt,m
1,2,t,jason,m
2,3,s,hila,f
3,4,w,matt,m
4,5,h,lashanda,f
5,6,z,randy,m
6,7,n,jon,m
7,8,a,stewart,m
8,9,c,solon,f
9,10,r,jeremy,m


In [ ]:
orders

,id,item,quantity_ordered,item_cost
0,1.0,playboy roll,1,12.00
1,1.0,rockstar roll,1,10.00
2,1.0,spider roll,1,8.00
3,1.0,keystone roll,1,25.00
4,1.0,salmon sashimi,6,3.00
5,1.0,tuna sashimi,6,2.50
6,1.0,edamame,1,6.00
7,2.0,salmon skin roll,1,8.00
8,2.0,playboy roll,1,12.00
9,2.0,unobtanium roll,1,35.00


In [ ]:
# 1 - Select all employees that are male

print("single table example # 1")
male_employees = pysqldf("SELECT * FROM employees WHERE gender = 'm'")
print(male_employees)

single table example # 1
    id lastname firstname gender
0    1        g      matt      m
1    2        t     jason      m
2    4        w      matt      m
3    6        z     randy      m
4    7        n       jon      m
5    8        a   stewart      m
6   10        r    jeremy      m
7   11        o       joe      m
8   12        j   jowanza      m
9   13        r      eric      m
10  15        p       tim      m
11  16        s      rudi      m
12  17        o     chris      m
13  18        i   michael      m
14  19        b      dima      m
15  20        f      drew      m


In [ ]:
# 2 - Get a count by first name
print("single table example # 2")
name_counts = pysqldf("SELECT firstname, COUNT (firstname) as occurances FROM employees GROUP BY firstname")
print(name_counts)

single table example # 2
   firstname  occurances
0      chris           1
1       dima           1
2       drew           1
3       eric           1
4       hila           1
5      jason           1
6     jeremy           1
7        joe           1
8        jon           1
9    jowanza           1
10  lashanda           1
11      matt           2
12   michael           1
13  michelle           1
14     randy           1
15      rudi           1
16     solon           1
17   stewart           1
18       tim           1


In [ ]:
# 3 - Get a count by first name, excluding non-employees
print("single table example # 3")
name_counts_emponly = pysqldf(""" SELECT firstname, COUNT(firstname) as occurances
                                FROM employees
                                WHERE firstname != 'rudi'
                                GROUP BY firstname""")
print(name_counts_emponly)

single table example # 3
   firstname  occurances
0      chris           1
1       dima           1
2       drew           1
3       eric           1
4       hila           1
5      jason           1
6     jeremy           1
7        joe           1
8        jon           1
9    jowanza           1
10  lashanda           1
11      matt           2
12   michael           1
13  michelle           1
14     randy           1
15     solon           1
16   stewart           1
17       tim           1


In [ ]:
# 4 - Use a case statement to define a new data column of california employees,
#    using "lower" to evaluate all names as lowercase to ensure case insensitivity
print("single table example # 4")
employees_cali = pysqldf("""SELECT *,
                        CASE
                          WHEN lower(firstname) = 'stewart' THEN 1
                          WHEN lower(firstname) = 'hila' THEN 1
                          WHEN lower(firstname) = 'jon' THEN 1
                          WHEN lower(firstname) = 'solon' THEN 1
                          ELSE 0
                        END as cali_emp
                        FROM employees
                        """)
print(employees_cali)

single table example # 4
    id lastname firstname gender  cali_emp
0    1        g      matt      m         0
1    2        t     jason      m         0
2    3        s      hila      f         1
3    4        w      matt      m         0
4    5        h  lashanda      f         0
5    6        z     randy      m         0
6    7        n       jon      m         1
7    8        a   stewart      m         1
8    9        c     solon      f         1
9   10        r    jeremy      m         0
10  11        o       joe      m         0
11  12        j   jowanza      m         0
12  13        r      eric      m         0
13  14        c  michelle      f         0
14  15        p       tim      m         0
15  16        s      rudi      m         0
16  17        o     chris      m         0
17  18        i   michael      m         0
18  19        b      dima      m         0
19  20        f      drew      m         0


In [ ]:
# 5 - Sort employees_cali by cali_emp descending, first name ascending (ascending is default)
print("single table example # 5")
employees_cali_sorted = pysqldf("""SELECT *,
                               CASE
                                  WHEN lower(firstname) = 'stewart' THEN 1
                                  WHEN lower(firstname) = 'hila' THEN 1
                                  WHEN lower(firstname) = 'jon' THEN 1
                                  WHEN lower(firstname) = 'solon' THEN 1
                                  ELSE 0
                                END as cali_emp
                                FROM employees
                                ORDER BY cali_emp DESC, firstname
                        """)
print(employees_cali_sorted)

single table example # 5
    id lastname firstname gender  cali_emp
0    3        s      hila      f         1
1    7        n       jon      m         1
2    9        c     solon      f         1
3    8        a   stewart      m         1
4   17        o     chris      m         0
5   19        b      dima      m         0
6   20        f      drew      m         0
7   13        r      eric      m         0
8    2        t     jason      m         0
9   10        r    jeremy      m         0
10  11        o       joe      m         0
11  12        j   jowanza      m         0
12   5        h  lashanda      f         0
13   1        g      matt      m         0
14   4        w      matt      m         0
15  18        i   michael      m         0
16  14        c  michelle      f         0
17   6        z     randy      m         0
18  16        s      rudi      m         0
19  15        p       tim      m         0


In [ ]:
##### Multi-Table operations ####

# 1.  Left join employees and orders table (keep all records from employees table,
#    matching records from orders)
print("multi table example # 1")
left_join = pysqldf("""SELECT *
                   FROM employees a
                   LEFT JOIN orders b ON a.id=b.id
                   WHERE a.firstname != 'rudi'
                   """)
print(left_join)

multi table example # 1
    id lastname firstname gender   id                  item  quantity_ordered  \
0    1        g      matt      m  1.0               edamame               1.0   
1    1        g      matt      m  1.0         keystone roll               1.0   
2    1        g      matt      m  1.0          playboy roll               1.0   
3    1        g      matt      m  1.0         rockstar roll               1.0   
4    1        g      matt      m  1.0        salmon sashimi               6.0   
5    1        g      matt      m  1.0           spider roll               1.0   
6    1        g      matt      m  1.0          tuna sashimi               6.0   
7    2        t     jason      m  2.0          playboy roll               1.0   
8    2        t     jason      m  2.0      salmon skin roll               1.0   
9    2        t     jason      m  2.0          tuna sashimi               4.0   
10   2        t     jason      m  2.0       unobtanium roll               1.0   
11  

In [ ]:
# 2.  "Right join" isn't supported in sqldf package, but switching order of tables and left join
#    is functionally equivalent
print("multi table example # 2")
right_join_equiv = pysqldf("""SELECT *
                            FROM orders b
                            LEFT JOIN employees a ON a.id=b.id
                          """)
print(right_join_equiv)

multi table example # 2
     id                     item  quantity_ordered  item_cost   id lastname  \
0   1.0             playboy roll                 1      12.00  1.0        g   
1   1.0            rockstar roll                 1      10.00  1.0        g   
2   1.0              spider roll                 1       8.00  1.0        g   
3   1.0            keystone roll                 1      25.00  1.0        g   
4   1.0           salmon sashimi                 6       3.00  1.0        g   
5   1.0             tuna sashimi                 6       2.50  1.0        g   
6   1.0                  edamame                 1       6.00  1.0        g   
7   2.0         salmon skin roll                 1       8.00  2.0        t   
8   2.0             playboy roll                 1      12.00  2.0        t   
9   2.0          unobtanium roll                 1      35.00  2.0        t   
10  2.0             tuna sashimi                 4       2.50  2.0        t   
11  2.0     yellowtail hand 

In [ ]:
# 3.  Inner join...select only records that match both tables
print("multi table example # 3")
inner_join = pysqldf("""SELECT *
                   FROM employees a
                    LEFT JOIN orders b ON a.id = b.id
                   WHERE a.id=b.id
                   """)
print(inner_join)

multi table example # 3
    id lastname firstname gender   id                  item  quantity_ordered  \
0    1        g      matt      m  1.0               edamame                 1   
1    1        g      matt      m  1.0         keystone roll                 1   
2    1        g      matt      m  1.0          playboy roll                 1   
3    1        g      matt      m  1.0         rockstar roll                 1   
4    1        g      matt      m  1.0        salmon sashimi                 6   
5    1        g      matt      m  1.0           spider roll                 1   
6    1        g      matt      m  1.0          tuna sashimi                 6   
7    2        t     jason      m  2.0          playboy roll                 1   
8    2        t     jason      m  2.0      salmon skin roll                 1   
9    2        t     jason      m  2.0          tuna sashimi                 4   
10   2        t     jason      m  2.0       unobtanium roll                 1   
11  

In [ ]:
# 4.  Matt G. sees bill, wonders how bill can be so low!
#    Join orders to employees, find who is ordering items less than 10 dollars, sorted by lowest cost
print("multi table example # 4")
inexpensive_items = pysqldf("""SELECT *
                         FROM orders a
                         LEFT JOIN employees b ON a.id= b.id
                         WHERE item_cost < 10
                         ORDER BY item_cost
                         """)
print(inexpensive_items)

multi table example # 4
     id                     item  quantity_ordered  item_cost   id lastname  \
0   NaN                diet coke                 1       1.95  NaN     None   
1   1.0             tuna sashimi                 6       2.50  1.0        g   
2   2.0             tuna sashimi                 4       2.50  2.0        t   
3   1.0           salmon sashimi                 6       3.00  1.0        g   
4   4.0            cucumber roll                 1       3.50  4.0        w   
5   4.0          california roll                 1       4.00  4.0        w   
6   1.0                  edamame                 1       6.00  1.0        g   
7   5.0               unagi roll                 1       6.50  5.0        h   
8   2.0     yellowtail hand roll                 1       7.00  2.0        t   
9   NaN  chicken teriaki hibachi                 1       7.95  NaN     None   
10  1.0              spider roll                 1       8.00  1.0        g   
11  2.0         salmon skin 

In [ ]:
# 4a.  Realizing some things are priced by piece, figure out who spent less than $20 on any one
#     type of food
print("multi table example # 4a")
inexpensive_line_items = pysqldf("""SELECT *,
                                (item_cost * quantity_ordered) as item_level_cost
                                 FROM orders a
                                 LEFT JOIN employees b ON a.id= b.id
                                 WHERE item_level_cost < 20
                                 ORDER BY item_level_cost
                                """)
print(inexpensive_line_items)

multi table example # 4a
     id                     item  quantity_ordered  item_cost   id lastname  \
0   NaN                diet coke                 1       1.95  NaN     None   
1   4.0            cucumber roll                 1       3.50  4.0        w   
2   4.0          california roll                 1       4.00  4.0        w   
3   1.0                  edamame                 1       6.00  1.0        g   
4   5.0               unagi roll                 1       6.50  5.0        h   
5   2.0     yellowtail hand roll                 1       7.00  2.0        t   
6   NaN  chicken teriaki hibachi                 1       7.95  NaN     None   
7   1.0              spider roll                 1       8.00  1.0        g   
8   2.0         salmon skin roll                 1       8.00  2.0        t   
9   5.0         firecracker roll                 1       9.00  5.0        h   
10  1.0            rockstar roll                 1      10.00  1.0        g   
11  2.0             tuna sa

In [ ]:
# 5.  Realizing that even item level cost is wrong question, Matt G. wants to know whose total lunch < $30
#    Need to use GROUP BY to get totals by person, then use HAVING instead of WHERE because
#    of the use of the GROUP BY summary function (WHERE is a record level operator)
print("multi table example # 5")
lunch_under_30 = pysqldf("""SELECT lastname, firstname,
                         SUM(item_cost * quantity_ordered) as lunch_cost
                         FROM orders a
                         LEFT JOIN employees b ON a.id= b.id
                         GROUP BY a.id
                         HAVING lunch_cost < 30
                         """)
print(lunch_under_30)

multi table example # 5
  lastname firstname  lunch_cost
0     None      None         9.9
1        w      matt         7.5


In [ ]:
# 6.  Matt G. wants to keep track of food consumption per person, particularly who the "lightweights"
#    are in the group.  Who's eating less than average on a cost basis?
#    This requires a subquery to first determine the average cost of this meal, passing that value
#    to the HAVING clause

# Subquery: returns a single value for the average lunch cost for employees (those with valid ID num)
# "SELECT SUM(item_cost * quantity_ordered)/COUNT(DISTINCT id) as avg_lunch_cost
# FROM orders WHERE id != 'NA'")
print("multi table example # 6")
lower_than_average_cost = pysqldf("""SELECT lastname, firstname,
                                  SUM(item_cost * quantity_ordered) as lunch_cost
                                  FROM orders a
                                  LEFT JOIN employees b ON a.id= b.id
                                  WHERE a.id != 'NA'
                                  GROUP BY a.id
                                  HAVING lunch_cost < (
                                                        SELECT SUM(item_cost * quantity_ordered)/COUNT(DISTINCT id) as avg_lunch_cost
                                                        FROM orders
                                                        WHERE id != 'NA'
                                                      )
                                """)

print(lower_than_average_cost)

multi table example # 6
  lastname firstname  lunch_cost
0        w      matt         7.5
1        h  lashanda        50.5


In [ ]:
# Print the first few rows of employees data
print("Employees Data:")
print(employees.head())

# Print the first few rows of orders data
print("Orders Data:")
print(orders.head())


Employees Data:
   id lastname firstname gender
0   1        g      matt      m
1   2        t     jason      m
2   3        s      hila      f
3   4        w      matt      m
4   5        h  lashanda      f
Orders Data:
    id            item  quantity_ordered  item_cost
0  1.0    playboy roll                 1       12.0
1  1.0   rockstar roll                 1       10.0
2  1.0     spider roll                 1        8.0
3  1.0   keystone roll                 1       25.0
4  1.0  salmon sashimi                 6        3.0


In [ ]:
# Inspect the columns of the employees and orders tables
print(employees.columns)
print(orders.columns)


Index(['id', 'lastname', 'firstname', 'gender'], dtype='object')
Index(['id', 'item', 'quantity_ordered', 'item_cost'], dtype='object')


In [ ]:
# Example of generalizing the query
lower_than_average_cost = pysqldf("""SELECT lastname, firstname,
                                  SUM(item_cost * quantity_ordered) as lunch_cost
                                  FROM orders a
                                  LEFT JOIN employees b ON a.id = b.id
                                  WHERE a.id IS NOT NULL
                                  GROUP BY a.id
                                  HAVING lunch_cost < (
                                                        SELECT SUM(item_cost * quantity_ordered)/COUNT(DISTINCT id) as avg_lunch_cost
                                                        FROM orders
                                                        WHERE id IS NOT NULL
                                                      )
                                """)
print(lower_than_average_cost)


  lastname firstname  lunch_cost
0        w      matt         7.5
1        h  lashanda        50.5


In [ ]:
# Example of displaying a cleaner output
print(left_join.head())


   id lastname firstname gender   id            item  quantity_ordered  \
0   1        g      matt      m  1.0         edamame               1.0   
1   1        g      matt      m  1.0   keystone roll               1.0   
2   1        g      matt      m  1.0    playboy roll               1.0   
3   1        g      matt      m  1.0   rockstar roll               1.0   
4   1        g      matt      m  1.0  salmon sashimi               6.0   

   item_cost  
0        6.0  
1       25.0  
2       12.0  
3       10.0  
4        3.0  


In [ ]:
# Query 1: Left join employees and orders table, keeping all employee records
left_join = pysqldf("""SELECT *
                   FROM employees a
                   LEFT JOIN orders b ON a.id = b.id
                   WHERE a.firstname != 'rudi'
                   """)
print(left_join)


    id lastname firstname gender   id                  item  quantity_ordered  \
0    1        g      matt      m  1.0               edamame               1.0   
1    1        g      matt      m  1.0         keystone roll               1.0   
2    1        g      matt      m  1.0          playboy roll               1.0   
3    1        g      matt      m  1.0         rockstar roll               1.0   
4    1        g      matt      m  1.0        salmon sashimi               6.0   
5    1        g      matt      m  1.0           spider roll               1.0   
6    1        g      matt      m  1.0          tuna sashimi               6.0   
7    2        t     jason      m  2.0          playboy roll               1.0   
8    2        t     jason      m  2.0      salmon skin roll               1.0   
9    2        t     jason      m  2.0          tuna sashimi               4.0   
10   2        t     jason      m  2.0       unobtanium roll               1.0   
11   2        t     jason   